# Alongshore transport

## First import some necessary packages

In [1]:
import logging
import pathlib
import sys
import warnings

import colorcet as cc
import dotenv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch

from bokeh.models import PanTool, WheelZoomTool, HoverTool
from bokeh.resources import INLINE
import bokeh.io

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [2]:
# # Read questions from cloud storage
questions = cd.read_questions(
    "./7_alongshore_transport.json"
    # "az://coastal-dynamics/questions/5_morphodynamics_upper_shoreface.json",
    # storage_options={"account_name": "coclico"},
)

# Alongshore sediment transport

Welcome to the notebook of week 6! The main topic of this notebook is cross-shore sediment transport. This notebook is relatively short. We will look into the CERC formula, and how to use it to obtain an (S, $\phi$-curve. Afterwards, we will use the (S, $\phi$-curve to determine coastal evolution around a breakwater.

## CERC formula and ($S$,$\phi$)-curve
Many different formulas exist to calculate bulk longshore sediment transport. One widely used formula is the CERC formula (Equation 8.4 in the book). We will use a version here which uses deep-water parameters. This formula, which is applicable for straight, parallel depth contours, is defined as:
$$S = \frac{K}{32(s-1)(1-p)} c_b \sin{2 \phi_0} H_0^2 $$
For a complete description of each parameter, see section 8.2.3 of the book.

For simplicity, we will assume that the offshore wave height $H_0$ is equal to the deep-water root-mean-square wave height $H_{rms,0}$, and can be represented by a single value of 1.8 m, with a period T of 6.5 s. This gives a value for $K$ of 0.7.

You might notice that the only variables left in the CERC formula are the breaking wave celerity $c_b$ and the offshore wave angle $\phi_0$. Starting from our values for $H_0$ and $T$, we can infer values for $c_b$ using linear wave theory with the dispersion relation. The function below does exactly that. Each step in the function is explained. Note that the value for $c_b$ depends on the angle of incidence.

In [3]:
# Maybe we don't distribute this function, and instead keep it private :-)

def disper(w, h, g=9.8):
    '''
    DISPER  Linear dispersion relation.
    
    absolute error in k*h < 5.0e-16 for all k*h
    
    Syntax:
    k = disper(w, h, [g])
    
    Input:
    w = 2*pi/T, were T is wave period
    h = water depth
    g = gravity constant
    
    Output:
    k = wave number
    
    Example
    k = disper(2*pi/5,5,g = 9.81);
    
    Copyright notice
    --------------------------------------------------------------------
    Copyright (C) 
    G. Klopman, Delft Hydraulics, 6 Dec 1994
    M. van der Lugt conversion to python, 11 Jan 2021
    
    '''    
    #make sure numpy array
    listType = type([1,2])
    Type = type(w)

    w = np.atleast_1d(w)
    
    #check to see if warning disappears
    wNul = w==0
    w[w==0] = np.nan

    
    w2 = w**2*h/g
    q = w2 / (1-np.exp(-w2**(5/4)))**(2/5)
    
    for j in np.arange(0,2):
        thq = np.tanh(q)
        thq2 = 1-thq**2
        aa = (1 - q*thq) * thq2
        
        #prevent warnings, we don't apply aa<0 anyway
        aa[aa<0] = 0
        
        bb = thq + q*thq2
        cc = q*thq - w2
        
        
        D = bb**2-4*aa*cc
        
        # initialize argument with the exception
        arg = -cc/bb
        
        # only execute operation on those entries where no division by 0 
        ix = np.abs(aa*cc)>=1e-8*bb**2 
        arg[ix] = (-bb[ix]+np.sqrt(D[ix]))/(2*aa[ix]) 

                
        q = q + arg

              
    k = np.sign(w)*q/h
    
    #set 0 back to 0
    k = np.where(wNul,0,k)

    #if input was a list, return also as list
    if Type==listType:
        k = list(k)
    elif len(k)==1:
        k = k[0]
        
    return k

In [4]:
def find_cb(phi0, H0, T, 
            g=9.81, gamma=0.8, hb=np.arange(0.1, 5.0, 0.01),
            print_report=False):
    """
    Returns breaking wave celerity cb [m/s] for given:
    - phi0 : angle of incidence [degrees]
    - H0   : deep water wave height [m]
    - T    : period [s]

    The parameter hb_guess is used as guessed values for the breaking depth. 
    From this array, the best-fitting value is chosen in the end. You can adjust this
    array to make estimates more accurate at the cost of computational efficiency. 
    """
    # First convert the angle of incidence to radians
    phi_rad = phi0 / 360 * 2 * np.pi
    
    # We start with calculating deep water celerity, wavelength, and angular frequency
    c0 = g * T / (2 * np.pi)
    L0 = c0 * T
    w  = T / (2 * np.pi)

    # For every value of hb_guess, the wavenumber k is determined using the dispersion relation
    k = disper(w, hb, g=g)  # Feel free to use your own implementation from week 2!

    # Next we calculate the celerity and group celerity for each breaking depth
    c = np.sqrt(g / k * np.tanh(k * hb))
    n = 1/2 * (1 + (2 * k * hb) / (np.sinh(2 * k * hb)))
    cg = n * c

    # In order to correctly shoal the waves, we also need the deep water group celerity
    n0 = 1/2
    cg0 = n0 * c0

    # And to account for refraction we need the angle of incidence at breaking
    phi = np.arcsin(np.sin(phi_rad) / c0 * c)
    
    # Shoaling & refraction coefficients
    Ksh = np.sqrt(cg0 / cg)
    Kref = np.sqrt(np.cos(phi_rad)/np.cos(phi))

    # Wave heights Hb at depth hb
    Hb = Ksh * Kref * H0

    # We are looking for an hb where the breaker parameter is 0.8
    # We can determine which value of hb in our array gets closest using the
    # following line of code:
    i = np.argmin(np.abs(Hb / hb - gamma))
    Hb_pred, hb_pred = Hb[i], hb[i]

    # Let's print what we found
    if print_report:
        print(f'predicted breaking depth: {hb_pred:.2f} m')
        print(f'predicted breaking wave height: {Hb_pred:.2f} m')
        print(f'gamma = {Hb_pred / hb_pred:.2f} [-]')

    # And finally return the associated value for cb
    return c[i]

Let's check that it works. The cell below shows that for an offshore wave height of 2 m, a period of 7 s, and a angle of incidence of 5 degrees, we get a $c_b$ of 4.92 m/s. You can check that this is in reasonable agreement with Example 8.1 in the book.

In [5]:
phi0 = 5  # degrees
H0 = 2  # m
T = 7  # s

print(f'cb: {find_cb(phi0, H0, T, print_report=True):.2f}')

predicted breaking depth: 2.79 m
predicted breaking wave height: 2.23 m
gamma = 0.80 [-]
cb: 4.92


Now that we have this function, we can calculate a breaking wave celerity for each angle of incidence.

In [6]:
# Set the range for which we want to calculate cb
phi_array = np.arange(-80, 80, 1)

# Initialize cb array
cb_array  = np.zeros(phi_array.shape)

# Loop through each phi and compute associated value for cb
for i in range(len(phi_array)):
    cb_array[i] = find_cb(phi_array[i], H0, T)

With cb, we can calculate our transport $S$ as a function of $\phi$, which means we can generate an ($S$,$\phi$)-curve! 

Remember that we can use the CERC formulation of Equation 8.10 for this. We use typical values of $K=0.7$, $p=0.4$, and $s=2.65$. 

Finish the equation below to compute the bulk sediment transport S.

In [7]:
def CERC(cb, phi0, H0,
         K=0.7,
         s=2.65,
         p=0.4
        ):
    
    S = K / (32 * (s - 1) * (1 - p)) * cb * np.sin(2*(phi0 / 360 * 2 * np.pi)) * H0**2

    return S

In [8]:
S = CERC(cb_array, phi_array, H0)

In [9]:
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

(hv.Curve((phi_array, S)) * hv.HLine(0).opts(color='black') * hv.VLine(0).opts(color='black')).opts(xlabel='angle [degrees]', ylabel='S [m3/s]', title='(S, phi)-curve', width=800, height=400, show_grid=True)

:Overlay
   .Curve.I :Curve   [x]   (y)
   .HLine.I :HLine   [x,y]
   .VLine.I :VLine   [x,y]

Using this plot and the code above, try to answer the questions below.

In [10]:
q1 = cd.QuestionFactory(questions["Q7-1"]).serve()
q2 = cd.QuestionFactory(questions["Q7-2"]).serve()
q3 = cd.QuestionFactory(questions["Q7-3"]).serve()
q4 = cd.QuestionFactory(questions["Q7-4"]).serve()

pn.Column(q1, q2, q3, q4)

Column
    [0] Column
        [0] StaticText(name='Q7-1', value='Rerun the code f...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q7-2', value='And for this same wave he...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q7-3', value='Change the offshore w...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q7-4', value='For which circumstances w...)
        [1] CheckBoxGroup(name='Options', options=['Increased porosity', ...])
        [2] Button(name='Submit')
        [3] StaticText()

This is the end of the first part of this notebook.

## Coastal evolution around a breakwater
We will now use the ($S$,$\phi$)-curve generated above to predict coastal evolution around a breakwater. Firstly, we need a wave climate. We use a simplified version of Table 8.2 from the book, with only two wave conditions:
* Condition 1: $H_0=1.2$ m with $T=8$ s and $\phi_0=40$ degrees. This condition occurs 15% of the time.
* Condition 2: $H_0=0.6$ m with $T=12$ and $\phi_0=-25$ degrees. This condition occurs 60% of the time.

15% of the time, no significant waves are present. Remember that positive angles result in positive transport (see also the image in Table 8.2).

Let's determine the average yearly transport for these conditions. For both conditions, we can determine the transport $S$.

In [11]:
# For condition 1
cb1 = find_cb(40, 1.2, 8)
S1  = CERC(cb1, 40, 1.2)

# For condition 2
cb2 = find_cb(-25, 0.6, 12)
S2  = CERC(cb2, -25, 0.6)

# These transports S are both given in m3/s. Remember that condition 1 occurs 15% of the year, and condition 2 occurs 60% of the year. For 1 year, the contributions are:
S_total = (0.15 * S1 + 0.6 * S2) * 365.25 * 24 * 3600

print(f'Contribution 1: {S1:.3f} m3/s')
print(f'Contribution 2: {S2:.3f} m3/s')
print(f'Total yearly transport: {S_total:.0f} m3/year')

Contribution 1: 0.124 m3/s
Contribution 2: -0.020 m3/s
Total yearly transport: 204961 m3/year


This total transport is positive, which means the cumulative transport is in positive direction. But what does this actually mean for the coastal morphology? According to Equation 8.16, we know the coastline will evolve by:
$$\frac{\partial Y}{\partial t} + \frac{1}{d} \frac{\partial S_x}{\partial x}=0$$
with $S_x$ the sediment transport in the $x$-direction. This equation means that the temporal evolution of the coastline position Y is determined by the *alongshore gradient* of the alongshore transport. For any location $x$ along the coastline, we can determine the coastline evolution through the following steps:
1) Determine the local angle $\phi$ of the coastline
2) Determine the sediment transport distribution of the coastline, which is a function of the local angle $\phi$
3) Determine the alongshore gradient of the sediment transport distribution
4) Determine the expected morphological change

When repeated, for example through a numerical model, this strategy can help predict morphological changes. This is useful for predicting the shoreline evolution around human interventions as well! Let us for example consider a breakwater perpendicular to the shore. We will try to use a numerical model to determine the shoreline evolution. Don't worry too much about the numerical model for now, we will focus on the concepts here! You will learn more about the numerical modelling part in the Coastal Modelling unit of the Coastal Engineering track.

Let's place the breakwater at $x=0$, and consider the domain $x \leq 0$.

Firstly, some assumptions need to be made. For simplicity, we will fix the depth of closure to a single value of $d=7$ m. Secondly, let's assume that the wave climate is given by the conditions described above (for which we already calculated the total yearly transport. We also need to impose initial and boundary conditions. These are thoroughly described by Equation 8.21 - 8.23. Briefly, for initial conditions, we assume a horizontal coastline (i.e $y=0$  along the coast). The following two boundary conditions are imposed:
* $S_x = S$, for $x=-\infty$ and for all $t$
* $S_x = 0$, for $x=0$ and for all $t$

With that, we are ready to do some modelling! We will look at a stretch of coast with a length of 10 km, with timesteps of 0.02 years.

**Note:** be careful with changing numerical parameters in the cell below. A relatively small time step is required to ensure numerical stability, changing it could lead to wiggles in the solution. Do feel free to adjust the end time (20 years)!

In [12]:
# numerical parameters
dx    = 100  # m
xmin  = -2500
dt    = 0.01  # year
tend  = 20  # year

# physical parameters
d     = 7 #  m

# space and time domain
X = np.arange(xmin, 0+dx, dx)
T = np.arange(0, tend, dt)

print("x-shape: ", X.shape)
print("t-shape: ", T.shape)

x-shape:  (26,)
t-shape:  (2000,)


We create a quick function to determine the transport as a function of the angle of incidence relative to the angle of the coast. The angle phi is defined as positive when it induces positive transport, the same as the image in Table 8.2.

In [13]:
def get_S(phi):
    """
    Returns yearly transport for angle phi [degrees]

    Transport is already scaled for the relative occurrence of the two conditions.
    """
    # For condition 1
    cb1 = find_cb(40 - phi, 1.2, 8)
    S1  = CERC(cb1, 40 - phi, 1.2)
    
    # For condition 2
    cb2 = find_cb(-25 - phi, 0.6, 12)
    S2  = CERC(cb2, -25 - phi, 0.6)
    
    return (0.15 * S1 + 0.6 * S2) * 365.25 * 24 * 3600

**Note:** You do not need to understand the function used for numerical modelling. However, each step of the process is described qualitatively, so it might be interesting to go through the function once.

In [14]:
# Maybe don't include numerical model function? Students will spend the first few weeks of Sierd's course on this, so probably don't give students this code :-)

def num_model_bw_updrift(X, T, d, print_info=False):

    dx = X[1] - X[0]  # m
    dt = T[1] - T[0]  # year

    # spatial values 
    phi  = np.zeros(X.shape)
    S    = np.zeros(X.shape)
    dSdX = np.zeros(X.shape)
    Y    = np.zeros((T.shape[0], X.shape[0]))  # 2D, to store all previous timesteps as well

    for it in range(len(T)):  # loop through all time steps

        for i in range(len(X)):  # loop through all locations to calculate alongshore transport

            angle = phi[i]
            S[i] = get_S(angle)

        S[-1] = 0  # set last value to 0 (boundary condition at breakwater)

        for i in range(1, len(X)-1):  # loop through all internal locations to calculate gradient alongshore transport

            dSdX[i] = (S[i+1] - S[i-1]) / (2 * dx)

        # Use up-/downwind scheme for boundaries
        dSdX[0]  = (S[1] - S[0]) / dx
        dSdX[-1] = (S[-1] - S[-2]) / dx

        dYdt = -1/d * dSdX  # from Equation 8.16
        Y[it] = Y[it-1] + dYdt * dt * 0.1  # superposition change with previous coastline

        for i in range(1, len(X)-1):  # loop through all internal locations to update the angles of the coastline
            phi[i] = np.arctan((Y[it, i+1] - Y[it, i-1]) / (2*dx)) * 360 / (2 * np.pi)
        phi[0] = np.arctan((Y[it, 1] - Y[it, 0]) / dx) * 360 / (2 * np.pi)
        phi[-1] = np.arctan((Y[it, -1] - Y[it, -2]) / dx) * 360 / (2 * np.pi)

    return Y

In [15]:
# Running this cell may take a minute. In order to ensure numerical stability for the shoreline angle we need a relatively small time step.

Y_t = num_model_bw_updrift(X, T, d)

Let's plot the results for a selection of years. You can modify which years to display if you want. Each line represents the coastline after a certain amount of years.

In [16]:
curve = hv.Curve(((),()))

years = [1, 2, 3, 4, 5, 20]
# years = np.arange(0, 21, 1)

for year in years:

    id = np.argmin(np.abs(T - year))
    
    curve *= hv.Curve((X, Y_t[id]), label=f'Year = {year}') 

breakwater = hv.Curve(((0,0),(0,100)), label='Breakwater').opts(color='black')
shoreline  = hv.Curve(((xmin,500),(0,0)), label='Shoreline').opts(color='#D2B48C')
(curve * breakwater * shoreline).opts(width=1200, height=400, legend_position='top_left')

:Overlay
   .Curve.I              :Curve   [x]   (y)
   .Curve.Year_equals_0  :Curve   [x]   (y)
   .Curve.Year_equals_1  :Curve   [x]   (y)
   .Curve.Year_equals_2  :Curve   [x]   (y)
   .Curve.Year_equals_3  :Curve   [x]   (y)
   .Curve.Year_equals_4  :Curve   [x]   (y)
   .Curve.Year_equals_5  :Curve   [x]   (y)
   .Curve.Year_equals_6  :Curve   [x]   (y)
   .Curve.Year_equals_7  :Curve   [x]   (y)
   .Curve.Year_equals_8  :Curve   [x]   (y)
   .Curve.Year_equals_9  :Curve   [x]   (y)
   .Curve.Year_equals_10 :Curve   [x]   (y)
   .Curve.Year_equals_11 :Curve   [x]   (y)
   .Curve.Year_equals_12 :Curve   [x]   (y)
   .Curve.Year_equals_13 :Curve   [x]   (y)
   .Curve.Breakwater     :Curve   [x]   (y)
   .Curve.Shoreline      :Curve   [x]   (y)

We see that the angle the shoreline makes with the breakwater is fairly constant throughout the years. This is the angle that leads to zero alongshore transport for the given conditions! As a final step, let us plot the transport as a function of the *shoreline orientation*.

**Note:** previously, we plotted the ($S$,$\phi$)-curve for a single wave condition only. We will now include multiple wave conditions (the two used above). This will result in a different curve, as you will see.

In [17]:
phi = np.linspace(-45, 120, 200)
S = np.zeros(phi.shape)

for i in range(len(phi)):
    S[i] = get_S(phi[i])

(hv.Curve((phi, S)) * hv.HLine(0).opts(color='black') * hv.VLine(0).opts(color='black')).opts(xlabel='angle [degrees]', ylabel='S [m3/year]', title='(S, phi)-curve', width=800, height=400, show_grid=True)

:Overlay
   .Curve.I :Curve   [x]   (y)
   .HLine.I :HLine   [x,y]
   .VLine.I :VLine   [x,y]

Using the plots and code, try to answer the questions below.

In [18]:
q5 = cd.QuestionFactory(questions["Q7-5"]).serve()
q6 = cd.QuestionFactory(questions["Q7-6"]).serve()
q7 = cd.QuestionFactory(questions["Q7-7"]).serve()
q8 = cd.QuestionFactory(questions["Q7-8"]).serve()

pn.Column(q5, q6, q7, q8)

Column
    [0] Column
        [0] StaticText(name='Q7-5', value='After how many y...)
        [1] RadioBoxGroup(name='Options', options=['7', '9', '11', '13'], value='7')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q7-6', value='The angle of the coastlin...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q7-7', value='Determine the e...)
        [1] FloatInput(name='Your Answer')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q7-8', value='Imagine we increase t...)
        [1] RadioBoxGroup(name='Options', options=['The shoreline angle clos...], value='The shoreline a...)
        [2] Button(name='Submit')
        [3] StaticText()

We have been working with very simple boundary conditions. We could improve this model further by using different boundary conditions. The questions below concern the improvement of boundary conditions.

In [19]:
q9 = cd.QuestionFactory(questions["Q7-9"]).serve()
q10 = cd.QuestionFactory(questions["Q7-10"]).serve()

pn.Column(q9, q10)

Column
    [0] Column
        [0] StaticText(name='Q7-9', value='Say we want t...)
        [1] CheckBoxGroup(name='Options', options=['reduce wave height for p...])
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q7-10', value='Say we want to include by...)
        [1] CheckBoxGroup(name='Options', options=['use a conditional bounda...])
        [2] Button(name='Submit')
        [3] StaticText()

We could extend this model with all sorts of additional functionality. Perhaps a first step would be to include the down-drift zone into the model. You will learn all about this in the Coastal Modelling unit! 

You have reached the end of this notebook.